- Stability AI
- stable-diffusion-img2img
- https://replicate.com/stability-ai/stable-diffusion-img2img/api

In [ ]:
!pip install replicate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for replicate: filename=replicate-0.8.1-py3-none-any.whl size=21099 sha256=ee535127104ea819608cab265ccee398f47c10020a19d671af30a3b732b0f682
  Stored in directory: /root/.cache/pip/wheels/7f/de/2f/7f55f2dcb401baa19b90823c42391b55b09a372f5751356b02
Successfully built replicate


In [ ]:
from getpass import getpass
import os
import shutil
import requests
import replicate
from google.colab import drive, files
drive.mount("/content/drive")

Mounted at /content/drive


#### Prepare Dataset

In [ ]:
os.makedirs("/content/origin_imgs", exist_ok=True)
os.makedirs("/content/gen_imgs", exist_ok=True)

In [ ]:
!unzip /content/drive/MyDrive/IN-ISP/roadcrack_300.zip -d /content/origin_imgs

In [ ]:
dataset_name = "roadcrack_300.zip"
dataset_path = os.path.join("/content/origin_imgs", os.path.splitext(dataset_name)[0])

In [ ]:
flist = os.listdir('/content/origin_imgs/roadcrack_300')
flist = [file for file in flist if 'crop_' in file]
flist = sorted(flist)
len(flist)

300

#### Get Model (Stable Diffusion - img2img)

In [ ]:
os.environ["REPLICATE_API_TOKEN"] = getpass("Enter your Replicate API token: ")

Enter your Replicate API token: ··········


In [ ]:
model = replicate.models.get("stability-ai/stable-diffusion-img2img")
version = model.versions.get("15a3689ee13b0d2616e98820eca31d4c3abcd36672df6afce5cb6feb1d66087d")

#### Generate Images

In [ ]:
flist[:10]

['crop_100.png',
 'crop_101.png',
 'crop_103.png',
 'crop_107.png',
 'crop_110.png',
 'crop_112.png',
 'crop_113.png',
 'crop_114.png',
 'crop_123.png',
 'crop_125.png']

In [ ]:
%%time

for fname in flist[:10]:
# for fname in flist:
    inputs = {
        'prompt': "fine road cracks, different background",
        'image': open(os.path.join(dataset_path, fname), "rb"),
        'prompt_strength': 0.13,
        'num_outputs': 8,
        'num_inference_steps': 30,
        'guidance_scale': 11,
        'scheduler': "K_EULER_ANCESTRAL",
        # 'seed': -2,
    }

    output = version.predict(**inputs)

    for i, url in enumerate(output):
        response = requests.get(url)
        with open("/content/gen_imgs/" + f"{os.path.splitext(fname)[0]}_gen_{i+1}.png", "wb") as f:
            f.write(response.content)

CPU times: user 2.12 s, sys: 191 ms, total: 2.32 s
Wall time: 2min 27s


#### Save to Google Drive

In [ ]:
gdrive_path = "/content/drive/MyDrive/IN-ISP"
zip_fname = "gen_imgs"

In [ ]:
shutil.make_archive(os.path.join(gdrive_path, zip_fname), "zip", "/content/gen_imgs")

'/content/drive/MyDrive/IN-ISP/gen_imgs.zip'